
## 1. Introduction and Overview
"""
Explain the problem you're solving. Why is it important?
What is the motivation behind choosing this project?
"""

## 2. Problem definition and objectives
"""
Describe your methodology.
Which algorithms, tools, or models are you using?
Include any architecture diagrams or flowcharts if applicable.
"""

## 3. Algorithm Design
"""
Analyze your algorithm's time and space complexity.
Use Big-O notation and explain under what conditions the performance changes.
"""

## 4. Correctness
"""
Describe the dataset(s) used.
Include source links, descriptions, and any preprocessing steps.
"""

## 5. Complexity Analysis
"""
Present the output of your experiments.
Discuss:
- Correctness
- Validation (cross-validation, test/train split, etc.)
- Performance (accuracy, precision, recall, etc.)
Include plots/tables where necessary.
"""

## 6. Implementation
"""
Discuss any difficulties encountered during the project.
What did you learn?
What would you do differently next time?
"""

## 7. Experiments,Datasets and Observations
TODO

## 8. Potential Issues
TODO

## 9. Conclusion and Improvements
TODO

### Appendix
TODO